**NOTE: This notebook is written for the Google Colab platform. However it can also be run (possibly with minor modifications) as a standard Jupyter notebook.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install parsley

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import parsley
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
#@title -- Auxiliary Functions -- { display-mode: "form" }
plot_kwds = dict(node_color='w',
                 node_size=2000, linewidths=1,
                 edgecolors='k', layout='dot')

def add_node(G, AST, labels, parent=None):
    if AST is None:
        return
    
    nid = len(G.nodes)
    
    if parent is None:
        G.add_node(nid)
    else:
        G.add_node(nid)
        G.add_edge(parent, nid)

    if isinstance(AST, tuple):
        labels[nid] = AST[0]
        for child in AST[1:]:
            add_node(G, child, labels, nid)
    elif isinstance(AST, list):
        labels[nid] = 'LIST'
        for child in AST:
            add_node(G, child, labels, nid)
    else:
        labels[nid] = AST

def draw(AST):
    G = nx.DiGraph()
    labels = {}
    add_node(G, AST, labels)
    G.graph['graph'] = {'rankdir': 'TD'}
    pos = nx.drawing.nx_pydot.graphviz_layout(G, prog='dot')
    return nx.draw(G, pos=pos, labels=labels, **plot_kwds)

def ensure_list(e):
    if isinstance(e, list):
        return e
    else:
        return [e]

## Languages, Grammars and Parsers

In the previous notebook we have explored regular expressions. Regular expression represent regular languages. However, these only represent one special type of a formal language. By a formal language $L$ we generally understand a **set of strings**  (sentences). We say that those sentences belong into language $L$. The strings are composed of symbols. The set $\Sigma$ of these symbols are called the **alphabet of language**  $L$ [[intro_langs]](#intro_langs).

### Describing a Language using the Set Builder Notation

Given that a formal language is actually represented by a set, the first way to describe a formal language is using set builder notation. If the alphabet of the language is e.g. $\Sigma = \{ a, b \}$, then the sentences will be composed of various combinations of characters $a$ and $b$ [[intro_langs]](#intro_langs). The language could then look e.g. as follows:

$$
L = \{ a, b, aa, bb \}.
$$
The empty string "" is, in the context of formal languages, usually denoted $\lambda$ [[intro_langs]](#intro_langs). So if we wanted to include the empty string in the language, we could write:

$$
L = \{ \lambda, a, b, aa, bb \}.
$$
When using se builder notation to represent a language, we can also express some allowed number of symbol repetitions etc., e.g.:

$$
L = \{ a^n b^n | n \geq 0 \}
$$
would express that characters $a$ and $b$ can repeat $n$ times, where $n \geq 0$ [[intro_langs]](#intro_langs).

### Describing Languages using Grammars

Describing formal languages using the builder set notation is definitely not the only option – there are several more. One very powerful way is to describe such language using a grammar. A grammar $G$ is an $n$-tuple $\langle V, T, S, P \rangle$, where [[intro_langs]](#intro_langs):

* $V$ is a finite set of variables;
* $T$ is a finite set of terminal symbols;
* $S \in V$ is the start variable;
* $P$ is a finite set of **production rules** .
Senteces that belong into language $L$ described by grammar $G$ are those that can be reached when starting from the start variable and applying the production rules an arbitrary number of times and in an arbitrary order – so that we get a sequences made solely of terminals.

The associated concepts will be best illustrated using a small example. Let us suppose that we have a grammar
$G = \langle \{ S \}, \{ a, b \}, S, P \rangle$, where $P$ contains two production rules [[intro_langs]](#intro_langs):

$$
\begin{aligned}
S &\rightarrow aSb \\
S &\rightarrow \lambda
\end{aligned}
$$
The rules express that we are allowed to replace $S$ using an empty string $\lambda$ or a combination $aSb$. So if we started from the start variable $S$ and applied the second production rule, we could conclude that language $L$ contains the empty string $\lambda$. If we first applied rule 1 and then rule 2, we would gain another valid sentence: $ab$. And we could go on in the same way, so the equivalent in the set builder notation would evidently be
$$
L = { a^n b^n | n \geq 0 },
$$

i.e. the language, which we have already encountered above.

#### Parsers

If we can define our language's grammar, there are tools, which can automatically turn it into a parser. A parser is able to verify that a certain sequence of symbols belongs into the language and it can also apply other necessary operations: for instance transform it into an abstract syntax tree – a representation that compilers work with.

#### Lexers

In the context of programming languages, parsers typically do not operate on raw text – the text is first preprocessed using a lexer, which splits it up into tokens correspoding e.g. to numeric literals, identifier etc.: lexers typically use regular expressions to achieve this.

### A Grammar for a Calculator; Parsley

If we intend to parse a larger amount of text, it is advisable to use tools such as Lex and Yacc or Flex and Bison, which, given the definition of tokens and the grammar, can automatically generate a fast lexer and parser in C or C++. For our (mainly pedagogical) purposes though, we will make use of a much simpler tool: the Python package `parsley`, which combines the functions of a lexer and a parser in a simple Python interface.

As an example we are going to define a grammar for a simple calculator.

#### Integers and Real Numbers

As our first step, we will write a grammar for parsing integers and real numbers. This is the part of the task that would traditionally be handled by a lexer. To create our grammar, we will use function `parsley.makeGrammar`, the input of which will be a string describing its rules. To be able to write multiline strings comfortably, we will wrap our string in three pairs of quotation marks. The `parsley` package allows us to match characters directly – in that case we will just need to wrap them in quotation marks in the rule – or to use one of the predefined basic rules [[parsley_ref]](#parsley_ref):

* **anything:**  matches a single character from the input;
* **letter:**  matches a single ASCII letter;
* **digit:**  matches a decimal digit;
* **letterOrDigit:**  combines the above;
* **end:**  matches the end of input;
* **ws:**  matches zero or more spaces, tabs, or newlines.
In a way similar to regular expressions, we can also define the number of repetitions using operators `+`, `*`, `{n}`, or alternatives using operator `|`. Other notation that is available can be found in [[parsley_ref]](#parsley_ref).

Let us first define rule `int`, which identifies an integer, i.e. a sequence of one or more digits and let us try to apply it to an example string:



In [ ]:
G = parsley.makeGrammar("""
int = digit+
""", {})

G("123").int()

As we can see, `parsley` turns our grammar's rules into Python function with the same names, which we can then call after supplying the grammar with some string. By default, a rule will yield the match with its last symbol as its return value. In the present case, this is the symbol `digit+`. Given that repetition is specified, our rule will return the entire matching sequence in a list.

Since we often need to get some part of the match back in the form of a string, `parsley` has special notation for this purpose: we only need to wrap the corresponding part of the rule in angled brackets:



In [ ]:
G = parsley.makeGrammar("""
int = <digit+>
""", {})

G("123").int()

And now the rule yields a string corresponding to our integer instead of a list. However, it would be better still to get the integer as a numeric data type directly. Fortunately, rules can include inline Python code, which allows us to process the return value in any way we like. It needs to be inserted at the end of the corresponding part of the rule and separated from it using `->`. In order to operate on the matches identified by the rule, we can name them by following them with a colon and an idetifier. In our case we could therefore convert the string into an integer using function `int` like this:



In [ ]:
G = parsley.makeGrammar("""
int = <digit+>:x -> int(x)
""", {})

G("123").int()

To explain further: statement `<digit+>:x` now assigns the corresponding match into variable `x` and statement `-> int(x)` specifies that the rule should return `int(x)`.

Let us further define a rule for real numbers: it will be similar to that for integers, except that the the input string will contain exactly one decimal point:



In [ ]:
G = parsley.makeGrammar("""
float = <digit+ '.' digit+>:x -> float(x)
""", {})

G("3.14").float()

In general, a grammar can, of course, contain more than one rule and the rules can refer to each other. Let us suppose that we already have rules `int` and `float` and we want to include a further rule `number` expressing that a number can be either an `int` pr a `float`:



In [ ]:
G = parsley.makeGrammar("""
number = float|int
int = <digit+>:x -> int(x)
float = <digit+ '.' digit+>:x -> float(x)
""", {})

print(G("123").number())
print(G("3.14").number())

#### Addition

As our next step, we will add support for addition. We are going to insert a new rule, which expresses that our language allows sentences where two numbers are connected using a `+` and that the return value should be the sum of the numbers in that case. As an alternative, we will also still allow a single number to be entered, therefore our new rule `addExpr` will have two alternative definitions, which we can express either using operator `|` or by providing two definitions of `addExpr` in two separate rows:



In [ ]:
G = parsley.makeGrammar("""
addExpr = number:x1 '+' number:x2 -> x1 + x2
addExpr = number
number = float|int
int = <digit+>:x -> int(x)
float = <digit+ '.' digit+>:x -> float(x)
""", {})

print(G("2+3").addExpr())
print(G("123").addExpr())

This grammar does not yet allow us to sum up more than two numbers – such sentences would not be valid under our language. However, we can include them by adding the following recursive rule:



In [ ]:
G = parsley.makeGrammar("""
addExpr = addExpr:x1 '+' addExpr:x2 -> x1 + x2
addExpr = number
number = float|int
int = <digit+>:x -> int(x)
float = <digit+ '.' digit+>:x -> float(x)
""", {})

print(G("2+3+4").addExpr())

#### Subtraction

Adding subtraction will be very easy. To make the notation simpler, we will move the actual calculations into a separate function `calc`, which will take the operator and two numbers as its inputs:



In [ ]:
def calc(op, x1, x2):
    if op == '+':
        return x1 + x2
    elif op == '-':
        return x1 - x2
    elif op == '*':
        return x1 * x2
    elif op == '/':
        return x1 / x2
    else:
        raise RuntimeError("Uknown operator '{}'.".format(op))

The subtraction operator then only need to be added as a further alternative in the rule. Naturally, we will also need to keep track of which operator was used so we will assign the match to variable `op`.

To be able to use a custom function within our grammar, we will need to register it using the dictionary argument of `parsley.makeGrammar`. We will state that our grammar's `'calc'` corresponds to the Python function `calc`, i.e. `{'calc': calc}`.



In [ ]:
G = parsley.makeGrammar("""
addExpr = addExpr:x1 ('+'|'-'):op addExpr:x2 -> calc(op, x1, x2)
addExpr = number
number = float|int
int = <digit+>:x -> int(x)
float = <digit+ '.' digit+>:x -> float(x)
""", {'calc': calc})

print(G("2+3-4").addExpr())

#### Multiplication and Division

It will be significantly more difficult to add multiplication and division. This is because of the conventional precedence rules: multiplication and division have higher priority than addition and subtraction. We can resolve that problem by adding a new type of expression (rule) called `multExpr`. We will take care when combining the rules and also make sure taht they are defined in the proper order. That way we can force all multiplications and divisions to be done before the results can be substituted back into `addExpr` rules.



In [ ]:
G = parsley.makeGrammar("""
expr = addExpr
addExpr = addExpr:x1 ('+' | '-'):op multExpr:x2 -> calc(op, x1, x2)
addExpr = multExpr
multExpr = multExpr:x1 ('*' | '/'):op multExpr:x2 -> calc(op, x1, x2)
multExpr = number
number = float|int
int = <digit+>:x -> int(x)
float = <digit+ '.' digit+>:x -> float(x)
""", {'calc': calc})

print(G("2*3+4").expr())
print(G("4+2*3").expr())

If this grammar encounters operator `+` (or `-`; rule `addExpr`) first, it cannot apply it directly, because the rule is defined to that one or both operands must match `multExpr`. Since this needs to be verified first, we can apply multiplications as part of that step.

#### Adding Brackets

If we want to specify the priority of operations explicitly, that can be done using brackets. So let us add them as a further alternative of the `multExpr` rule:



In [ ]:
G = parsley.makeGrammar("""
expr = addExpr
addExpr = addExpr:x1 ('+' | '-'):op multExpr:x2 -> calc(op, x1, x2)
addExpr = multExpr
multExpr = multExpr:x1 ('*' | '/'):op multExpr:x2 -> calc(op, x1, x2)
multExpr = number
multExpr = '(' addExpr:x ')' -> x
number = float|int
int = <digit+>:x -> int(x)
float = <digit+ '.' digit+>:x -> float(x)
""", {'calc': calc})

print(G("2*(3+4)").expr())
print(G("(3+4)*2").expr())

### Abstract Syntax Trees

In practice, we often need to apply relatively complex analyses onto the results of parsing. In the context of compiler, for instance, optimization is done before translating into machine code. This is why parsers usually transform source codes into so called abstract syntax trees, which are easier to analyze.

For illustration, we will now rewrite our calculator grammar so that instead of doing computations directly, it will create an abstract syntax tree which we are then going to visualize. We will do this by substituting operation `calc` with a triple  `(op, x1, x2)` consisting of the operator `op` and the left and the right operand. This is going to yield a simple tree composed of the nested triples.



In [ ]:
G = parsley.makeGrammar("""
expr = addExpr
addExpr = ( addExpr:x1 ('+' | '-'):op multExpr:x2 -> (op, x1, x2)
          | multExpr)
multExpr = ( multExpr:x1 ('*' | '/'):op number:x2 -> (op, x1, x2)
           | number | '(' expr:x ')' -> x)
number = float|int
int = <digit+>:x -> int(x)
float = <digit+ '.' digit+>:x -> float(x)
""", {})

AST = G("(2*2+3*2)*4").expr()
print(AST)
draw(AST)

### Parsing HTML

In the notebook on regular expressions, we have shown how to remove HTML tags from a text. However, we have concluded tha regular expressions are not expressive enough to remove HTML tags including their content. This is because they cannot express recursive patterns and they would not allow us to track the opening and closing of tags.

In our next example, we are going to define a grammar for HTML parsing and display the resulting abstract syntax tree. To simplify things, we are going to assume that all tags are going to be paired tags (we could, of course, handle unpaired tags as well, but for our purposes, that would be an unnecessary complication).

#### Text Except Characters `<` and `>`

As our first rule, we will create one that matches any text except characters `<` and `>` that denote tags. To do this, we are going to use the specil notation `?(condition)`, which allows us to evaluate a match using inline Python code.



In [ ]:
G = parsley.makeGrammar("""
except_angle = :x ?(not x in '<>')
text = <except_angle+>
""", {})

print(G("abcd").text())

#### The Opening and the Closing Tag

Next we are going to create rules for the opening and the closing tag:



In [ ]:
G = parsley.makeGrammar("""
except_angle = :x ?(not x in '<>')
closing_tag = '</' <except_angle+>:c '>' -> c
opening_tag = '<' <except_angle+>:c '>' -> c
""", {})

print(G("<tag>").opening_tag())
print(G("</tag>").closing_tag())

#### Tag Pairs

Next we are going to define tag pairs. A pair is composed of an opening and a closing tags and both tags are required to match. The tags can enclose some text or another tag.



In [ ]:
G = parsley.makeGrammar("""
except_angle = :x ?(not x in '<>')
closing_tag = '</' <except_angle+>:c '>' -> c
opening_tag = '<' <except_angle+>:c '>' -> c
expr = opening_tag:t1 expr:e closing_tag:t2 ?(t1 == t2) -> ('<' + t1 + '>', e)
expr = <except_angle+>
""", {})

print(G("<b>content</b>").expr())
print(G("<b><div>content</div></b>").expr())

#### Chaining Several Expressions

In general, tags can also be surrounded by text or other tags and similarly, tags can contain a combination of multiple other tags and text. We will therefore create rule `expr = expr:e1 expr:e2`, which will allow expressions to be chained. We will also use an auxiliary function `ensure_list`, which will (when necessary) wrap an expression in a list and the join the lists together. As a result, all expression (whether tags or text) at the same level will be concatenated into a single list.



In [ ]:
text = """
text above
<div>
div 1 content
<span>inner span 1</span>
<span>inner span 2</span>
</div><div>
div 2 content
<span>inner span</span>
</div>
text below
"""

In [ ]:
G = parsley.makeGrammar("""
except_angle = :x ?(not x in '<>')
closing_tag = '</' <except_angle+>:c '>' -> c
opening_tag = '<' <except_angle+>:c '>' -> c
expr = opening_tag:t1 expr:e closing_tag:t2 ?(t1 == t2) -> ('<' + t1 + '>', e)
expr = expr:e1 expr:e2 -> ensure_list(e1) + ensure_list(e2)
expr = <except_angle+> -> "txt"
""", {'ensure_list': ensure_list})

AST = G(text).expr()
plt.figure(figsize=(10, 5))
draw(AST)

#### Removing HTML Tags Including Their Content

As our final step, we are going to modify our grammar to meet the task from the previous notebook. We will remove all HTML tags including the content.



In [ ]:
text = """
text above
<div>
div 1 content
<span>inner span 1</span>
<span>inner span 2</span>
</div><div>
div 2 content
<span>inner span</span>
</div>
text below
"""

In [ ]:
G = parsley.makeGrammar("""
except_angle = :x ?(not x in '<>')
closing_tag = '</' <except_angle+>:c '>' -> c
opening_tag = '<' <except_angle+>:c '>' -> c
expr = opening_tag:t1 expr:e closing_tag:t2 ?(t1 == t2) -> ('')
expr = expr:e1 expr:e2 -> e1 + e2
expr = <except_angle+>
""", {})

print(G(text).expr())

---
### Task: Parse a List

**The cell below, defines variable `text`, which contains the textual representation of a list. Write grammar `G` with the main rule being named `list` that will parse `text` back into a list of strings.** 

---


In [ ]:
lst = ["abcd", "efg", "hij", "klmn"]
text = ",".join(lst)
print(text)

In [ ]:
G = parsley.makeGrammar("""


# ---


""", {})

#### Testing

Now let's test whether the list gets parsed correctly.



In [ ]:
l = G(text).list()

if l == lst:
    print("List parsed correctly.")
else:
    print("List parsed correctly.")
    print("-- Expected: {}".format(l))
    print("-- Got: {}".format(lst))

### References

<a id="intro_langs">[intro_langs]</a> Linz, P., 2006. An introduction to formal languages and automata. Jones & Bartlett Learning.

<a id="parsley_ref">[parsley_ref]</a> Parsley Reference. URL: <https://parsley.readthedocs.io/en/latest/reference.html>

